# 梧桐杯 AI 安全告警智能研判系统 - V3 增强版（冲击 99%）

## 🎯 V3 改进点

1. ✨ **数据增强**：11,000 → 30,000+ 条（2.7x）
2. ✨ **Stacking 集成**：替代简单 Voting，学习各模型优势
3. ✨ **10 折交叉验证**：更稳定，减少过拟合
4. ✨ **超参数优化**：更深的树、更多迭代

## 📋 训练流程

1. ✅ 克隆 GitHub 仓库
2. ✅ 安装依赖
3. ✅ 检查 GPU
4. ✅ 检查数据
5. ✅ 数据预处理
6. ✅ **数据增强（V3 核心）**
7. ✅ 训练 V3 模型
8. ✅ 查看结果
9. ✅ 下载模型

## 🎯 目标

- **准确率目标**: 99%+
- **训练时间**: 约 20-30 分钟
- **模型版本**: v3.0.0

## ⚙️ 运行前准备

1. 确保运行时类型设置为 **GPU**（Runtime > Change runtime type > GPU）
2. 按顺序执行每个单元格

---

In [1]:
# 1. 克隆 GitHub 仓库
!git clone https://github.com/alltobebetter/WuTong.git
%cd WuTong

# 查看项目结构
!ls -la

Cloning into 'WuTong'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 99 (delta 26), reused 95 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (99/99), 1.59 MiB | 5.14 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/WuTong
total 192
drwxr-xr-x 9 root root  4096 Feb 14 14:57 .
drwxr-xr-x 1 root root  4096 Feb 14 14:57 ..
-rw-r--r-- 1 root root 13448 Feb 14 14:57 colab_train.ipynb
-rw-r--r-- 1 root root 98655 Feb 14 14:57 colab_train_ok.ipynb
-rw-r--r-- 1 root root 16155 Feb 14 14:57 colab_train_v3.ipynb
drwxr-xr-x 4 root root  4096 Feb 14 14:57 data
drwxr-xr-x 2 root root  4096 Feb 14 14:57 doc
drwxr-xr-x 8 root root  4096 Feb 14 14:57 .git
-rw-r--r-- 1 root root   244 Feb 14 14:57 .gitignore
drwxr-xr-x 2 root root  4096 Feb 14 14:57 legacy
drwxr-xr-x 3 root root  4096 Feb 14 14:57 models
-rw-r--r-- 1 root root  4454 Feb 14 14:57 README.md
-rw-r--r-- 1 root root 

In [2]:
# 2. 安装依赖
print("📦 安装依赖包...")
!pip install -q -r requirements.txt

print("\n✅ 依赖安装完成！")

# 验证关键包
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
import pandas as pd

print(f"XGBoost 版本: {xgb.__version__}")
print(f"CatBoost 版本: {cb.__version__}")
print(f"LightGBM 版本: {lgb.__version__}")
print(f"Pandas 版本: {pd.__version__}")

📦 安装依赖包...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.9 MB/s eta 0:00:00

✅ 依赖安装完成！
XGBoost 版本: 3.1.3
CatBoost 版本: 1.2.8
LightGBM 版本: 4.6.0
Pandas 版本: 2.2.2


In [3]:
# 3. 检查 GPU 可用性
import torch

if torch.cuda.is_available():
    print(f"✅ GPU 可用: {torch.cuda.get_device_name(0)}")
    print(f"   显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ GPU 不可用，将使用 CPU 训练（速度较慢）")
    print("   建议: Runtime > Change runtime type > GPU")

✅ GPU 可用: Tesla T4
   显存: 14.56 GB


In [4]:
# 4. 检查数据集
import pandas as pd
from pathlib import Path

data_files = list(Path('data').rglob('*.xlsx'))
print(f"找到 {len(data_files)} 个数据文件:")
for f in data_files:
    print(f"  - {f}")

if data_files:
    df = pd.read_excel(data_files[0])
    print(f"\n数据集大小: {len(df)} 条")
    print(f"\n攻击类型分布:")
    print(df.iloc[:, -1].value_counts())
else:
    print("\n⚠️ 未找到数据文件！")

找到 2 个数据文件:
  - data/原始告警信息样例数据集V1.2.xlsx
  - data/raw/原始告警信息样例数据集V1.2.xlsx

数据集大小: 11000 条

攻击类型分布:
attack_type(结果列，需参赛选手判断输出)
正常访问              3413
目录遍历攻击             972
远程命令执行攻击           965
SQL注入攻击            964
文件包含攻击             959
XSS跨站脚本攻击          958
CSRF攻击             942
Java反序列化漏洞利用攻击     920
文件上传攻击             907
Name: count, dtype: int64


In [5]:
# 5. 数据预处理
print("🔄 开始数据预处理...")

import glob
excel_files = glob.glob('data/**/*.xlsx', recursive=True)

if excel_files:
    data_file = excel_files[0]
    print(f"使用数据文件: {data_file}")
    !python scripts/ingest.py "{data_file}"
    print("\n✅ 数据预处理完成！")

    parquet_files = glob.glob('data/staging/*.parquet')
    print(f"\n生成的 parquet 文件: {parquet_files}")
else:
    print("❌ 未找到 Excel 数据文件！")

🔄 开始数据预处理...
使用数据文件: data/原始告警信息样例数据集V1.2.xlsx
2026-02-14 14:58:22,618 INFO 读取 原始告警信息样例数据集V1.2.xlsx 成功，形状: (11000, 8)
2026-02-14 14:58:22,977 INFO 已保存标准化数据: /content/WuTong/data/staging/原始告警信息样例数据集V1.2.parquet  (11000 条)
✔ 已生成: /content/WuTong/data/staging/原始告警信息样例数据集V1.2.parquet

✅ 数据预处理完成！

生成的 parquet 文件: ['data/staging/原始告警信息样例数据集V1.2.parquet']


In [6]:
# 6. 数据增强（V3 核心 - 冲击 99%）
print("="*60)
print("🚀 V3 数据增强开始")
print("="*60)
print("\n策略:")
print("  - SQL 注入: 关键词替换、注释变换、空格编码")
print("  - XSS 攻击: 标签大小写、事件处理器、编码变换")
print("  - 命令注入: 分隔符变换、命令组合")
print("  - URL 路径: 大小写、编码、路径分隔符")
print("\n目标: 11,000 → 30,000+ 条 (2.7x)\n")

!python scripts/augment_data.py --target-size 30000 --ratio 2.5

print("\n" + "="*60)
print("✅ 数据增强完成！")
print("="*60)

# 查看增强后的数据
import pandas as pd
import glob

augmented_files = glob.glob('data/staging/*augmented*.parquet')
if augmented_files:
    df_aug = pd.read_parquet(augmented_files[0])
    print(f"\n📊 增强后统计:")
    print(f"  总数据量: {len(df_aug)} 条")
    print(f"  增强倍数: {len(df_aug) / 11000:.2f}x")
    print(f"\n类别分布:")
    print(df_aug['attack_type'].value_counts())

🚀 V3 数据增强开始

策略:
  - SQL 注入: 关键词替换、注释变换、空格编码
  - XSS 攻击: 标签大小写、事件处理器、编码变换
  - 命令注入: 分隔符变换、命令组合
  - URL 路径: 大小写、编码、路径分隔符

目标: 11,000 → 30,000+ 条 (2.7x)

2026-02-14 14:58:24,464 INFO === 数据增强开始 ===
2026-02-14 14:58:24,465 INFO 输入文件: /content/WuTong/data/staging/原始告警信息样例数据集V1.2.parquet
2026-02-14 14:58:24,465 INFO 目标数据量: 30000
2026-02-14 14:58:24,552 INFO 原始数据量: 11000
2026-02-14 14:58:24,561 INFO 原始类别分布:
attack_type
正常访问              3413
目录遍历攻击             972
远程命令执行攻击           965
SQL注入攻击            964
文件包含攻击             959
XSS跨站脚本攻击          958
CSRF攻击             942
Java反序列化漏洞利用攻击     920
文件上传攻击             907
Name: count, dtype: int64
2026-02-14 14:58:25,253 INFO   处理进度: 2000 / 11000
2026-02-14 14:58:25,912 INFO   处理进度: 4000 / 11000
2026-02-14 14:58:26,519 INFO   处理进度: 6000 / 11000
2026-02-14 14:58:27,101 INFO   处理进度: 8000 / 11000
2026-02-14 14:58:27,730 INFO   处理进度: 10000 / 11000
2026-02-14 14:58:28,720 INFO 
=== 数据增强完成 ===
2026-02-14 14:58:28,720 INFO 原始数据量: 11000
2026-02-14 1

In [7]:
# 7. 训练 V3 模型（Stacking + 10折CV）
print("="*60)
print("🚀 V3 模型训练开始")
print("="*60)
print("\n配置:")
print("  - 模型: XGBoost + CatBoost + LightGBM")
print("  - 集成方式: Stacking (Logistic Regression)")
print("  - 交叉验证: 10 折")
print("  - 超参数: 优化版（更深的树、更多迭代）")
print("  - 预计时间: 20-30 分钟\n")

!python scripts/train_v3.py --version v3.0.0 --cv-splits 10

print("\n" + "="*60)
print("✅ V3 训练完成！")
print("="*60)

🚀 V3 模型训练开始

配置:
  - 模型: XGBoost + CatBoost + LightGBM
  - 集成方式: Stacking (Logistic Regression)
  - 交叉验证: 10 折
  - 超参数: 优化版（更深的树、更多迭代）
  - 预计时间: 20-30 分钟

使用数据: /content/WuTong/data/staging/原始告警信息样例数据集V1.2_augmented.parquet
模型版本: v3.0.0
交叉验证: 是 (10 折)
Stacking: 是
2026-02-14 14:58:32,371 INFO === 训练开始 V3 (冲击 99%): version=v3.0.0 ===
2026-02-14 14:58:32,401 INFO 数据加载: 29258 条
2026-02-14 14:58:32,404 INFO 攻击类型分布:
attack_type
正常访问              4080
SQL注入攻击           4068
XSS跨站脚本攻击         3872
远程命令执行攻击          3138
目录遍历攻击            2916
文件包含攻击            2877
CSRF攻击            2826
Java反序列化漏洞利用攻击    2760
文件上传攻击            2721
Name: count, dtype: int64
2026-02-14 14:58:32,405 INFO 特征提取 ...
2026-02-14 14:58:32,405 INFO 开始特征提取，共 29258 条记录 ...
2026-02-14 14:58:33,013 INFO   特征提取进度: 2000 / 29258
2026-02-14 14:58:33,494 INFO   特征提取进度: 4000 / 29258
2026-02-14 14:58:33,989 INFO   特征提取进度: 6000 / 29258
2026-02-14 14:58:34,834 INFO   特征提取进度: 8000 / 29258
2026-02-14 14:58:35,613 INFO   特征提取进度: 1000

In [8]:
# 7-alternative. 快速训练（不带交叉验证，约 10-15 分钟）
# 如果想快速测试，可以运行这个单元格代替上面的单元格

# print("⚡ 快速训练模式（不带交叉验证）...")
# !python scripts/train_v3.py --version v3.0.0-fast --no-cv
# print("\n✅ 快速训练完成！")

In [9]:
# 8. 查看 V3 训练结果
import json
from pathlib import Path

manifest_path = Path('models/v3.0.0/manifest.json')

if manifest_path.exists():
    with open(manifest_path, 'r', encoding='utf-8') as f:
        manifest = json.load(f)

    print("="*70)
    print("📊 V3 训练结果")
    print("="*70)
    print(f"\n版本: {manifest['version']}")
    print(f"训练时间: {manifest['trained_at']}")
    print(f"数据量: {manifest['data_rows']} 条")
    print(f"特征数: {len(manifest['feature_list'])} 个")
    print(f"类别数: {len(manifest['classes'])} 类")

    config = manifest.get('training_config', {})
    print(f"\n训练配置:")
    print(f"  - 交叉验证: {config.get('n_cv_splits', 'N/A')} 折")
    print(f"  - 集成方式: {config.get('use_stacking', False) and 'Stacking' or 'Voting'}")

    print("\n" + "="*70)
    print("🎯 模型性能")
    print("="*70)

    metrics = manifest['metrics']

    # 单模型性能
    for model_name in ['xgboost', 'catboost', 'lightgbm']:
        if model_name in metrics:
            m = metrics[model_name]
            print(f"\n{model_name.upper()}:")
            print(f"  测试准确率: {m['test_accuracy']:.4f} ({m['test_accuracy']*100:.2f}%)")
            print(f"  测试 F1: {m['test_f1']:.4f}")
            if m.get('cv_accuracy'):
                print(f"  CV 准确率: {m['cv_accuracy']:.4f} (±{m.get('cv_std', 0):.4f})")

    # 集成模型性能
    if 'ensemble' in metrics:
        e = metrics['ensemble']
        print(f"\n{'='*70}")
        print(f"🏆 V3 集成模型（最终模型）")
        print(f"{'='*70}")
        print(f"  准确率: {e['test_accuracy']:.4f} ({e['test_accuracy']*100:.2f}%)")
        print(f"  F1 分数: {e['test_f1']:.4f}")
        print(f"  集成方式: {e.get('ensemble_type', 'N/A')}")

        # 与 V2 对比
        v2_acc = 0.9759  # V2 的准确率
        improvement = (e['test_accuracy'] - v2_acc) * 100
        print(f"\n  📈 相比 V2 提升: {improvement:+.2f}%")

        if e['test_accuracy'] >= 0.99:
            print(f"\n  🎉 恭喜！达到 99% 目标！")
        elif e['test_accuracy'] >= 0.985:
            print(f"\n  ✨ 非常接近 99%，表现优秀！")

    print("\n" + "="*70)

    # 详细分类报告
    report_path = Path('models/v3.0.0/classification_report.txt')
    if report_path.exists():
        print("\n📋 详细分类报告:")
        print("="*70)
        with open(report_path, 'r', encoding='utf-8') as f:
            print(f.read())
else:
    print("❌ 未找到 V3 训练结果文件！")
    print("请先运行训练单元格")

📊 V3 训练结果

版本: v3.0.0
训练时间: 2026-02-14T15:12:26.217893+00:00
数据量: 29258 条
特征数: 25 个
类别数: 9 类

训练配置:
  - 交叉验证: 10 折
  - 集成方式: Stacking

🎯 模型性能

XGBOOST:
  测试准确率: 0.9831 (98.31%)
  测试 F1: 0.9831
  CV 准确率: 0.9839 (±0.0000)

CATBOOST:
  测试准确率: 0.9706 (97.06%)
  测试 F1: 0.9708
  CV 准确率: 0.9749 (±0.0000)

LIGHTGBM:
  测试准确率: 0.9851 (98.51%)
  测试 F1: 0.9852
  CV 准确率: 0.9850 (±0.0000)

🏆 V3 集成模型（最终模型）
  准确率: 0.9836 (98.36%)
  F1 分数: 0.9836
  集成方式: stacking

  📈 相比 V2 提升: +0.77%


📋 详细分类报告:
                precision    recall  f1-score   support

        CSRF攻击     0.9439    0.9522    0.9480       565
Java反序列化漏洞利用攻击     1.0000    1.0000    1.0000       552
       SQL注入攻击     0.9732    0.9816    0.9774       814
     XSS跨站脚本攻击     0.9974    0.9871    0.9922       775
        文件上传攻击     1.0000    1.0000    1.0000       544
        文件包含攻击     0.9964    0.9757    0.9859       575
          正常访问     0.9609    0.9632    0.9621       816
        目录遍历攻击     0.9983    1.0000    0.9991       583
      远程命令

In [10]:
# 9. 对比 V2 vs V3
print("📊 模型版本对比\n")
!python scripts/compare_models.py

📊 模型版本对比

找到 2 个模型版本: v3.0.0, v1.0.0

+-------------+---------------------+----------+-------------+----------+----------+
| 版本        | 训练时间            | 数据量   | 模型类型    |   准确率 | F1分数   |
+=============+=====================+==========+=============+==========+==========+
| v1.0.0      | 2026-02-14T13:20:01 | 11000    | RF+GB       |   0.9755 | N/A      |
+-------------+---------------------+----------+-------------+----------+----------+
| v3.0.0      | 2026-02-14T15:12:26 | 29258    | XGB+Cat+LGB |   0.9836 | 0.9836   |
+-------------+---------------------+----------+-------------+----------+----------+
| └─ XGBoost  |                     |          |             |   0.9831 | 0.9831   |
+-------------+---------------------+----------+-------------+----------+----------+
| └─ CatBoost |                     |          |             |   0.9706 | 0.9708   |
+-------------+---------------------+----------+-------------+----------+----------+
| └─ LightGBM |                     |        

In [11]:
# 10. 打包 V3 模型文件
print("📦 打包 V3 模型文件...")

!zip -r models_v3.0.0.zip models/v3.0.0/

print("\n✅ 打包完成！")
print("\n文件大小:")
!ls -lh models_v3.0.0.zip

📦 打包 V3 模型文件...
  adding: models/v3.0.0/ (stored 0%)
  adding: models/v3.0.0/aggregator.pkl (deflated 70%)
  adding: models/v3.0.0/catboost.pkl (deflated 88%)
  adding: models/v3.0.0/ensemble.pkl (deflated 79%)
  adding: models/v3.0.0/lightgbm.pkl (deflated 63%)
  adding: models/v3.0.0/feature_list.json (deflated 59%)
  adding: models/v3.0.0/label_encoder.pkl (deflated 16%)
  adding: models/v3.0.0/manifest.json (deflated 55%)
  adding: models/v3.0.0/classification_report.txt (deflated 55%)
  adding: models/v3.0.0/xgboost.pkl (deflated 68%)

✅ 打包完成！

文件大小:
-rw-r--r-- 1 root root 24M Feb 14 15:12 models_v3.0.0.zip


In [12]:
# 11. 下载 V3 模型到本地
from google.colab import files

print("⬇️ 开始下载 V3 模型文件...")
print("（下载完成后解压到本地项目的 models/ 目录）\n")

files.download('models_v3.0.0.zip')

print("\n✅ 下载完成！")
print("\n📝 后续步骤:")
print("1. 解压 models_v3.0.0.zip")
print("2. 将 models/v3.0.0/ 目录复制到本地项目")
print("3. 运行推理: python scripts/infer.py data/xxx.xlsx --model-version v3.0.0")
print("4. V3 模型在 CPU 上运行完全没问题！")

⬇️ 开始下载 V3 模型文件...
（下载完成后解压到本地项目的 models/ 目录）



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ 下载完成！

📝 后续步骤:
1. 解压 models_v3.0.0.zip
2. 将 models/v3.0.0/ 目录复制到本地项目
3. 运行推理: python scripts/infer.py data/xxx.xlsx --model-version v3.0.0
4. V3 模型在 CPU 上运行完全没问题！


---

## ✅ V3 训练完成！

### 🎯 V3 改进总结

| 项目 | V2 | V3 | 提升 |
|------|----|----|------|
| 数据量 | 11,000 | 30,000+ | 2.7x |
| 集成方式 | Voting | Stacking | ✨ |
| 交叉验证 | 5 折 | 10 折 | ✨ |
| 超参数 | 基础 | 优化 | ✨ |
| 预期准确率 | 97.5% | 99%+ | +1.5%+ |

### 🔑 关键改进

1. **数据增强**（最重要）
   - SQL 注入：关键词替换、注释变换
   - XSS 攻击：标签大小写、事件处理器
   - 命令注入：分隔符变换
   - URL 路径：编码变换、路径分隔符

2. **Stacking 集成**
   - 学习各模型的优势领域
   - 对难分类样本（如 CSRF）效果更好

3. **超参数优化**
   - 树深度：8 → 10
   - 迭代次数：200 → 300
   - 学习率：0.1 → 0.05

### 📦 V3 模型文件

```
models/v3.0.0/
├── ensemble.pkl              # Stacking 集成模型
├── xgboost.pkl               # XGBoost (300 棵树)
├── catboost.pkl              # CatBoost (300 轮)
├── lightgbm.pkl              # LightGBM (300 棵树)
├── label_encoder.pkl         # 标签编码器
├── aggregator.pkl            # 告警聚合器
├── manifest.json             # 模型元数据
├── feature_list.json         # 特征列表
└── classification_report.txt # 详细分类报告
```

### 💡 使用建议

1. **本地推理**：V3 模型在 CPU 上运行完全没问题
2. **API 服务**：直接替换 V2 模型即可
3. **比赛提交**：只提交模型文件，不包含原始数据

### 🚀 如果还想提升

1. **补充公开数据集**：CSIC 2010、CICIDS2017
2. **针对 CSRF 优化**：单独训练二分类器
3. **特征工程**：N-gram、熵值、编码检测

---

**项目地址**: https://github.com/alltobebetter/WuTong

**祝比赛顺利！冲击 99%！** 🎉
